In [152]:
import requests
import sys, os
from dotenv import load_dotenv
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)
from security_routes.utilities import hash_password
from pathlib import Path

## Base URL

You can run this notebook locally, on main runner, or development runner, please uncomment the one you are going to use, and comment the other base url's

In [153]:
# Main runner
# base_url = "https://tmv9bz5v4q.us-east-1.awsapprunner.com/"
# Development runner
base_url = "https://cp3by92k8p.us-east-1.awsapprunner.com/"
# Local
# base_url = "http://localhost:8000/"
prefix = "latest"

In [154]:
hash_password("password")

'$2b$12$BtDqUNK8Kiq1RSWaMsvbVu7bPOfobUGD1otoi5WfTTkgkGLO1XGqm'

In [155]:
load_dotenv("../.env", override=True)

True

In [156]:
adminpassword = os.getenv("ADMIN_PASSWORD")
adminpassword is not None

True

In [157]:
response = requests.post(
    base_url+"latest/token", data={"username": "admin", "password": adminpassword}
)
admin_token = response.json()["access_token"]

In [112]:
len(admin_token)

145

Create user and group

In [113]:
new_user_data = {
        "username": "new_user",
        "email": "new_user@example.com",
        "is_admin": False,
    }

new_auth_data = {
    "username": "new_user",
    "password": "password123",
}

response = requests.post(
        base_url + f"{prefix}/users",
        params=new_user_data,
        data=new_auth_data,
        headers={"Authorization": f"Bearer {admin_token}"},
    )

response

<Response [400]>

In [114]:
response.text

'{"detail":"Username already registered"}'

In [115]:
group_data = {
    "name": "new_group",
    "description": "A new group",
}
response = requests.post(
    base_url+f"{prefix}/groups",
    params=group_data,
    headers={"Authorization": f"Bearer {admin_token}"},
)
response

<Response [400]>

In [116]:
response.text

'{"detail":"Group already exists"}'

In [117]:
link_data = {
    "username": "new_user",
    "groupname": "new_group",
}
response = requests.post(
    base_url+f"{prefix}/link-user-group",
    params=link_data,
    headers={"Authorization": f"Bearer {admin_token}"},
)
response

<Response [400]>

In [118]:
response.text

'{"detail":"User is already linked to this group"}'

Create a revision

In [119]:
response = requests.post(
    base_url+"latest/token", data={"username": "new_user", "password": "password123"}
)
token = response.json()["access_token"]

In [120]:
len(token)

151

In [121]:
new_version_data = {
    "name": "New Version",
    "iso_language": "eng",
    "iso_script": "Latn",
    "abbreviation": "NV",
    "rights": "Some Rights",
    "machineTranslation": False,
    "is_reference": True,
}

response = requests.post(
    base_url+f"{prefix}/version",
    json=new_version_data,
    headers={"Authorization": f"Bearer {token}"},
)
version_id =response.json()["id"]
response.json()

{'id': 1727,
 'name': 'New Version',
 'iso_language': 'eng',
 'iso_script': 'Latn',
 'abbreviation': 'NV',
 'rights': 'Some Rights',
 'forward_translation_id': None,
 'back_translation_id': None,
 'machineTranslation': False,
 'owner_id': 382,
 'group_ids': [],
 'is_reference': True}

Update Name

In [122]:
# Modify name of the version
attr_update = {
    "id": version_id,
    "name": "New Version 2"
    }
response = requests.put(
    base_url+f"{prefix}/version",
    json=attr_update,
    headers={"Authorization": f"Bearer {token}"},
)
version_name = response.json()["name"]
version_name

'New Version 2'

In [123]:
print(new_version_data)

{'name': 'New Version', 'iso_language': 'eng', 'iso_script': 'Latn', 'abbreviation': 'NV', 'rights': 'Some Rights', 'machineTranslation': False, 'is_reference': True}


In [124]:
response.text

'{"name":"New Version 2","iso_language":"eng","abbreviation":"NV","forward_translation_id":null,"machine_translation":false,"deleted":false,"owner_id":382,"id":1727,"iso_script":"Latn","rights":"Some Rights","back_translation_id":null,"is_reference":true,"deletedAt":null}'

In [125]:
headers = {"Authorization": f"Bearer {admin_token}"}

In [126]:

test_revision = {
    "version_id": version_id,
    "name": "Test Revision",
}
test_upload_file = Path("../fixtures/uploadtest.txt")

with open(test_upload_file, "rb") as file:
    files = {"file": file}
    response = requests.post(
    base_url + f"{prefix}/revision", params=test_revision, files=files, headers=headers
    )
revision_id =response.json()["id"]
response.json()

{'id': 2652,
 'bible_version_id': 1727,
 'version_abbreviation': 'NV',
 'date': '2025-02-10',
 'name': 'Test Revision',
 'published': False,
 'back_translation_id': None,
 'machineTranslation': False,
 'iso_language': 'eng',
 'is_reference': False}

In [127]:
response.text

'{"id":2652,"bible_version_id":1727,"version_abbreviation":"NV","date":"2025-02-10","name":"Test Revision","published":false,"back_translation_id":null,"machineTranslation":false,"iso_language":"eng","is_reference":false}'

In [128]:
response = requests.get(
    base_url+f"{prefix}/revision",
    headers={"Authorization": f"Bearer {token}"},
)
response.json()

[{'id': 1802,
  'bible_version_id': 1172,
  'version_abbreviation': 'NV',
  'date': '2024-10-02',
  'name': 'Test Revision',
  'published': False,
  'back_translation_id': None,
  'machineTranslation': False,
  'iso_language': 'eng',
  'is_reference': False},
 {'id': 1799,
  'bible_version_id': 1168,
  'version_abbreviation': 'NV',
  'date': '2024-10-02',
  'name': 'Test Revision',
  'published': False,
  'back_translation_id': None,
  'machineTranslation': False,
  'iso_language': 'eng',
  'is_reference': False},
 {'id': 1877,
  'bible_version_id': 1212,
  'version_abbreviation': 'NV',
  'date': '2024-10-02',
  'name': 'Test Revision',
  'published': False,
  'back_translation_id': None,
  'machineTranslation': False,
  'iso_language': 'eng',
  'is_reference': False},
 {'id': 1878,
  'bible_version_id': 1212,
  'version_abbreviation': 'NV',
  'date': '2024-10-02',
  'name': 'Test Revision',
  'published': False,
  'back_translation_id': None,
  'machineTranslation': False,
  'iso_lang

In [129]:
response = requests.get(
    base_url+f"{prefix}/version",
    headers={"Authorization": f"Bearer {token}"},
)
response.json()

[{'id': 1168,
  'name': 'New Version 2',
  'iso_language': 'eng',
  'iso_script': 'Latn',
  'abbreviation': 'NV',
  'rights': 'Some Rights',
  'forward_translation_id': None,
  'back_translation_id': None,
  'machineTranslation': False,
  'owner_id': 382,
  'group_ids': [473],
  'is_reference': True},
 {'id': 1169,
  'name': 'New Version',
  'iso_language': 'eng',
  'iso_script': 'Latn',
  'abbreviation': 'NV',
  'rights': 'Some Rights',
  'forward_translation_id': None,
  'back_translation_id': None,
  'machineTranslation': False,
  'owner_id': 382,
  'group_ids': [473],
  'is_reference': True},
 {'id': 1170,
  'name': 'New Version 2',
  'iso_language': 'eng',
  'iso_script': 'Latn',
  'abbreviation': 'NV',
  'rights': 'Some Rights',
  'forward_translation_id': None,
  'back_translation_id': None,
  'machineTranslation': False,
  'owner_id': 382,
  'group_ids': [473],
  'is_reference': True},
 {'id': 1171,
  'name': 'New Version 2',
  'iso_language': 'eng',
  'iso_script': 'Latn',
  '

Create a new assesment

In [130]:
assessment_ids = []

assessment_data = ["semantic-similarity", "word-alignment", "sentence-length"]
for assessment in assessment_data:
    data = {"revision_id": revision_id, "reference_id": revision_id, "type": str(assessment)}
    response = requests.post(f"{base_url}v3/assessment", params=data, headers=headers)

    print(response.status_code)
    assessment_ids.append(response.json()[0]["id"])
    print(response.json())
    print(response.text)

200
[{'id': 4056, 'revision_id': 2652, 'reference_id': 2652, 'type': 'semantic-similarity', 'status': 'queued', 'requested_time': '2025-02-10T16:36:18.650733', 'start_time': None, 'end_time': None, 'owner_id': 115}]
[{"id":4056,"revision_id":2652,"reference_id":2652,"type":"semantic-similarity","status":"queued","requested_time":"2025-02-10T16:36:18.650733","start_time":null,"end_time":null,"owner_id":115}]
200
[{'id': 4057, 'revision_id': 2652, 'reference_id': 2652, 'type': 'word-alignment', 'status': 'queued', 'requested_time': '2025-02-10T16:36:20.747111', 'start_time': None, 'end_time': None, 'owner_id': 115}]
[{"id":4057,"revision_id":2652,"reference_id":2652,"type":"word-alignment","status":"queued","requested_time":"2025-02-10T16:36:20.747111","start_time":null,"end_time":null,"owner_id":115}]
200
[{'id': 4058, 'revision_id': 2652, 'reference_id': 2652, 'type': 'sentence-length', 'status': 'queued', 'requested_time': '2025-02-10T16:36:21.364032', 'start_time': None, 'end_time': 

In [136]:

for assessment in assessment_ids:
    assessment_id =  assessment
    response_get = requests.get(
        base_url+f"{prefix}/result",
        headers={"Authorization": f"Bearer {admin_token}"},
        params={"assessment_id": assessment_id},
    )
    print(response_get.json())

{'results': [{'id': 61606752, 'assessment_id': 4056, 'vref': 'GEN 1:1', 'source': None, 'target': None, 'score': 0.9999999403953552, 'flag': False, 'note': None, 'revision_text': None, 'reference_text': None, 'hide': False}, {'id': 61606753, 'assessment_id': 4056, 'vref': 'GEN 1:2', 'source': None, 'target': None, 'score': 1.0, 'flag': False, 'note': None, 'revision_text': None, 'reference_text': None, 'hide': False}, {'id': 61606754, 'assessment_id': 4056, 'vref': 'GEN 1:3', 'source': None, 'target': None, 'score': 1.0, 'flag': False, 'note': None, 'revision_text': None, 'reference_text': None, 'hide': False}], 'total_count': 3}
{'results': [{'id': 61606749, 'assessment_id': 4057, 'vref': 'GEN 1:1', 'source': None, 'target': None, 'score': 0.7131190708705357, 'flag': False, 'note': None, 'revision_text': None, 'reference_text': None, 'hide': False}, {'id': 61606750, 'assessment_id': 4057, 'vref': 'GEN 1:2', 'source': None, 'target': None, 'score': 0.8043554687500001, 'flag': False, 'n

In [140]:
data = {"revision_id": revision_id, "reference_id": revision_id, "type":"ngrams"}
response = requests.post(f"{base_url}v3/assessment", params=data, headers=headers)




In [141]:
print(response.status_code)
print(response.json())
print(response.text)
id = response.json()[0]["id"]
assessment_ids.append(id)

200
[{'id': 4059, 'revision_id': 2652, 'reference_id': 2652, 'type': 'ngrams', 'status': 'queued', 'requested_time': '2025-02-10T16:42:15.763301', 'start_time': None, 'end_time': None, 'owner_id': 115}]
[{"id":4059,"revision_id":2652,"reference_id":2652,"type":"ngrams","status":"queued","requested_time":"2025-02-10T16:42:15.763301","start_time":null,"end_time":null,"owner_id":115}]


In [143]:
assessment_id =  id
response_get = requests.get(
    base_url+f"{prefix}/result",
    headers={"Authorization": f"Bearer {admin_token}"},
    params={"assessment_id": assessment_id},
)
print(response_get.json())

{'results': [], 'total_count': 0}


In [144]:
for assessment in assessment_ids:
    assessment_id =  assessment
    response_delete = requests.delete(
        f"{base_url}v3/assessment",
        headers={"Authorization": f"Bearer {admin_token}"},
        params={"assessment_id": assessment_id},
    )
    print(response_delete.json())


{'detail': 'Assessment 4056 deleted successfully'}
{'detail': 'Assessment 4057 deleted successfully'}
{'detail': 'Assessment 4058 deleted successfully'}
{'detail': 'Assessment 4059 deleted successfully'}


Delete version and revision

In [145]:
response = requests.delete(
    base_url+f"{prefix}/revision",
    params={"id": revision_id},
    headers={"Authorization": f"Bearer {admin_token}"},
)
response.json()

{'detail': 'Revision 2652 deleted successfully.'}

In [146]:
response = requests.delete(
    base_url+f"{prefix}/version",
    params={"id": version_id},
    headers={"Authorization": f"Bearer {admin_token}"},
)
response.json()

{'detail': 'Version New Version 2 successfully deleted.'}

Delete user and group

In [ ]:
# response = requests.delete(
#     base_url+f"{prefix}/users",
#     params={"username": "new_user"},
#     headers={"Authorization": f"Bearer {admin_token}"},
# )
# response

<Response [500]>

In [148]:
# response.text

'Internal Server Error'

In [149]:
# response = requests.delete(
#     base_url+f"{prefix}/groups",
#     params={"groupname": "new_group"},
#     headers={"Authorization": f"Bearer {admin_token}"},
# )
# response

<Response [400]>

In [150]:
# response.text

'{"detail":"Group is linked to users and cannot be deleted"}'

In [151]:
# headers = {"Authorization": f"Bearer {admin_token}"}
# response = requests.get(f"{base_url}v3/language", headers=headers)
# assert response.status_code == 200
# assert {"iso639": "eng", "name": "english"} in response.json()
# print(response.json())